In [1]:
import pandas as pd
import numpy as np
import random
import os
import subprocess
import shutil

from tqdm import tqdm
from biopandas.pdb import PandasPdb
from torch_geometric.data import HeteroData

from scipy.spatial.transform import Rotation
import Bio.PDB

from Bio.Data.IUPACData import protein_letters_3to1

from collections import defaultdict

from sklearn.neighbors import kneighbors_graph as knn_graph


C:\Users\Пуся\AppData\Local\Temp\ipykernel_12652\3203096695.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import statsmodels.datasets.utils as du

### Чтение данных из PDB-файла 

In [3]:
LIG = 'ligand'
REC = 'receptor'

In [4]:
class Atom: 

    def __init__(self, name, coords):
        self.name = name 
        self.x_coord, self.y_coord, self.z_coord = coords
        self.element = 'C' if self.name == 'CA' or self.name == 'C'else 'N'

    def get_coords(self): 
        return np.array([self.x_coord, self.y_coord, self.z_coord])

In [5]:
class Chain: 

    def __init__(self, id, res_name, res_seq):
        self.id = id
        self.atoms = []
        self.res_name = res_name
        self.res_seq = res_seq

    def add_atom(self, name, coords):
        self.atoms.append(Atom(name, coords))

    def __len__(self):
        return len(self.atoms)
    
    def get_vectors_of_coords(self):

        n_c, ca_c, c_c = 0, 0, 0

        for atom in self.atoms:
            if atom.name == 'N':
                n_c = atom.get_coords()
            elif atom.name == 'CA':
                ca_c = atom.get_coords()
            else: 
                c_c = atom.get_coords() 


        u_i = (n_c - ca_c) / np.linalg.norm(n_c - ca_c)
        t_i = (c_c - ca_c) / np.linalg.norm(c_c - ca_c)
        n_i = np.cross(u_i, t_i) / np.linalg.norm(np.cross(u_i, t_i))
        v_i = np.cross(n_i, u_i)

        
        return n_i, u_i, v_i
        
    


In [106]:
class MyObject: 

    def __init__(self, type_):
        self.chains = []
        self.type = type_ 
        self.edge_index = None
        self.n_coords, self.c_coords, self.ca_coords = [], [], []


    def add_chain(self, chain: Chain):
        self.chains.append(chain)

    def get_coords_arrays(self):

        for chain in self.chains:
            for atom in chain.atoms: 
                if atom.name == 'CA':
                    self.ca_coords.append(atom.get_coords())
                elif atom.name == 'C':
                    self.c_coords.append(atom.get_coords())
                else:
                    self.n_coords.append(atom.get_coords())

    def __len__(self):
        return len(self.n_coords)


    def get_vectors_of_coords_arrays(self):

        n_i_list, u_i_list, v_i_list = [], [], []

        for chain in self.chains:
            n_i, u_i, v_i = chain.get_vectors_of_coords()
            n_i_list.append(n_i)
            u_i_list.append(u_i)
            v_i_list.append(v_i)
            
        return np.stack(n_i_list), np.stack(u_i_list), np.stack(v_i_list)


In [30]:
class Tasks:

    def __init__(self):
        self.tasks = []

    def add_task(self, paths, type_:str):
        self.tasks.append((paths, type_))

    def get_task(self, idx: int):
        if idx < len(self.tasks):
            return self.tasks[idx]
        else:
            raise IndexError()

In [109]:
class Loader:

    def __init__(self, data_file, knn_size=3, split = ',', verbose=False):
        self.parser = Bio.PDB.PDBParser()
        self.knn_size = knn_size
        self.data_to_load = pd.read_csv(data_file, sep=split, header=0)
        print(self.data_to_load)
        self.verbose = verbose
        self.tasks = []

    def parse_pdb(self, file_path, type_) -> MyObject:

        structure = self.parser.get_structure(file_path, file_path)
        myObject = MyObject(type_)
        
        for model in structure:
            for one_chain in model:
                for part in one_chain.get_residues():

                    chain = Chain(
                        id=repr(one_chain).replace("<Chain id=", "").replace(">", ""), 
                        res_name=part.get_resname(), 
                        res_seq=str(part).split(' ')[4].split('=')[1])
                    
                    for atom in part: 
                        if atom.get_name() in ['C', 'CA', 'N']:
                            chain.add_atom(atom.get_name(), atom.get_coord())
                            
                    if len(chain) != 0:
                        myObject.add_chain(chain)
            
        return myObject

    def read_data(self) -> Tasks:
        tasks = Tasks()
        for id, line in self.data_to_load.iterrows():
            path_to_rec = './structures/' + line['path'] + '_r_b.pdb'
            path_to_lig = './structures/' + line['path']  + '_l_b.pdb'
            tasks.add_task((path_to_lig, path_to_rec), line['split'])
        self.tasks = tasks
        return tasks
    
    def process_data(self, indexes):
        pairs = []
        
        for idx in indexes: 
            paths, type_ = self.tasks.get_task(idx)
            ligand = self.parse_pdb(paths[0], LIG)
            receptor = self.parse_pdb(paths[1], REC)
            ligand.get_coords_arrays()
            receptor.get_coords_arrays()
            edge_index = knn_graph(np.vstack(( ligand.c_coords, ligand.ca_coords, ligand.n_coords )), self.knn_size)
            ligand.edge_index = edge_index
            edge_index = knn_graph(np.vstack(( receptor.c_coords, receptor.ca_coords, receptor.n_coords )), self.knn_size)
            receptor.edge_index = edge_index
            pairs.append((ligand, receptor))
        
        return pairs

    

In [114]:
loader = Loader('./splits_test.csv')
tasks = loader.read_data()
print(tasks.get_task(0))
print(loader.process_data([0])[0][0].edge_index)


   path split
0  1A2K  test
(('./structures/1A2K_l_b.pdb', './structures/1A2K_r_b.pdb'), 'test')
  (0, 393)	1.0
  (0, 196)	1.0
  (0, 197)	1.0
  (1, 394)	1.0
  (1, 197)	1.0
  (1, 198)	1.0
  (2, 395)	1.0
  (2, 198)	1.0
  (2, 199)	1.0
  (3, 396)	1.0
  (3, 199)	1.0
  (3, 200)	1.0
  (4, 397)	1.0
  (4, 200)	1.0
  (4, 201)	1.0
  (5, 398)	1.0
  (5, 201)	1.0
  (5, 202)	1.0
  (6, 399)	1.0
  (6, 202)	1.0
  (6, 398)	1.0
  (7, 400)	1.0
  (7, 203)	1.0
  (7, 399)	1.0
  (8, 401)	1.0
  :	:
  (579, 382)	1.0
  (580, 187)	1.0
  (580, 384)	1.0
  (580, 188)	1.0
  (581, 188)	1.0
  (581, 385)	1.0
  (581, 384)	1.0
  (582, 189)	1.0
  (582, 386)	1.0
  (582, 385)	1.0
  (583, 190)	1.0
  (583, 387)	1.0
  (583, 191)	1.0
  (584, 191)	1.0
  (584, 388)	1.0
  (584, 387)	1.0
  (585, 192)	1.0
  (585, 389)	1.0
  (585, 193)	1.0
  (586, 193)	1.0
  (586, 390)	1.0
  (586, 194)	1.0
  (587, 194)	1.0
  (587, 391)	1.0
  (587, 390)	1.0
